In [ ]:
import os

os.environ["API_KEY"] = "your_api_key_here"  # Set your API key here

# Example: Accessing it right after setting
print(os.environ["API_KEY"])


XokeN0fDHyRtoZaB9HbVXCNd3uv5Piw4


In [2]:
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime, timedelta
import time
import concurrent.futures
from functools import partial

#############################################################################
# 1. Fetch S&P 500 Tickers
#############################################################################
def get_sp500_tickers():
    """
    Retrieves the list of S&P 500 tickers from Wikipedia.
    Returns a list of ticker symbols.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    df = tables[0]
    tickers = df["Symbol"].tolist()
    # Replace '.' with '-' if needed
    tickers = [t.replace(".", "-") for t in tickers]
    return tickers

#############################################################################
# 2. Fetch Price Data from FMP
#############################################################################
def get_price_data_fmp(ticker, apikey, from_date="1900-01-01", to_date=None):
    """
    Fetch daily adjusted close data for `ticker` from FinancialModelingPrep.
    Returns a sorted DataFrame with 'Adj Close' among other columns.

    Requirements:
      - A valid FMP API key.
      - Ticker symbol, e.g. "AAPL".
      - from_date / to_date in "YYYY-MM-DD" format.
    """
    if to_date is None:
        to_date = datetime.today().strftime("%Y-%m-%d")
    url = (
        f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}"
        f"?from={from_date}&to={to_date}&apikey={apikey}"
    )
    resp = requests.get(url)
    if resp.status_code != 200:
        raise Exception(f"API request for {ticker} failed with status code {resp.status_code}")
    data = resp.json()
    if "historical" not in data:
        raise Exception(f"No 'historical' data found for {ticker}.")

    df = pd.DataFrame(data["historical"])
    df["date"] = pd.to_datetime(df["date"])
    df.set_index("date", inplace=True)
    df.sort_index(inplace=True)
    # Rename if needed
    if "adjClose" in df.columns:
        df.rename(columns={"adjClose": "Adj Close"}, inplace=True)
    if "Adj Close" not in df.columns:
        raise ValueError(f"No 'Adj Close' in the data for {ticker}.")

    return df

#############################################################################
# 3. Ornstein-Uhlenbeck Fit
#############################################################################
def fit_ou_mle(X, dt=1.0):
    """
    Fit an Ornstein–Uhlenbeck process by closed-form MLE:
       dX = kappa*(mu - X)*dt + sigma*dW

    Returns (kappa, mu, sigma, half_life).
    Raises ValueError if not mean-reverting (Phi out of (0,1)).
    """
    X = np.asarray(X)
    N = len(X)
    if N < 2:
        raise ValueError("Need at least 2 data points for OU model.")

    X_t   = X[:-1]
    X_tp1 = X[1:]
    N_float = float(N - 1)

    S_x   = np.sum(X_t)
    S_y   = np.sum(X_tp1)
    S_xx  = np.sum(X_t * X_t)
    S_xy  = np.sum(X_t * X_tp1)

    num = (N_float * S_xy) - (S_x * S_y)
    den = (N_float * S_xx) - (S_x * S_x)
    Phi = num / den

    if not (0 < Phi < 1):
        raise ValueError(f"Estimated Phi={Phi:.4f} out of (0,1). Not mean-reverting.")

    kappa = -np.log(Phi) / dt
    mu    = (S_y - Phi*S_x) / (N_float*(1 - Phi))

    a = mu*(1 - Phi)
    resid = X_tp1 - a - Phi*X_t
    var_res = np.sum(resid**2) / N_float

    sigma_sq = 2*kappa*var_res / (1 - Phi**2)
    if sigma_sq < 0:
        raise ValueError(f"Negative sigma^2 estimated ({sigma_sq:.4f}).")
    sigma = np.sqrt(sigma_sq)

    half_life = np.log(2)/kappa
    return kappa, mu, sigma, half_life

def detrend_and_fit_ou(price_series):
    """
    1) Fit linear regression: Price(t) = intercept + slope*t
    2) residual = Price - (intercept + slope*t)
    3) Fit OU to the residual.

    Returns a dict with:
      intercept, slope, r2_trend, residual
      ou_kappa, ou_mu, ou_sigma, ou_half_life
      or ou_error if fails
    """
    N = len(price_series)
    t_idx = np.arange(N)

    X_lr = sm.add_constant(t_idx)
    lr = sm.OLS(price_series.values, X_lr).fit()
    intercept, slope = lr.params
    fitted_trend = intercept + slope*t_idx
    residual = price_series.values - fitted_trend

    out = {
        "intercept": intercept,
        "slope": slope,
        "r2_trend": lr.rsquared,
        "residual": residual
    }

    try:
        kappa, mu_resid, sigma, hl = fit_ou_mle(residual, dt=1.0)
        out["ou_kappa"] = kappa
        out["ou_mu"] = mu_resid
        out["ou_sigma"] = sigma
        out["ou_half_life"] = hl
    except ValueError as e:
        out["ou_error"] = str(e)

    return out

#############################################################################
# 4. Momentum Filter & Backtest
#############################################################################
def compute_momentum(price_series, window=5):
    """
    Simple momentum measure:
    MOM_t = (P_t - P_{t-window}) / P_{t-window}
    """
    return (price_series - price_series.shift(window)) / price_series.shift(window)

def backtest_mr_with_momentum(price_series, residual, mu_resid, sigma,
                              z_thresh=2.0, mom_window=5, mom_thresh=0.0):
    """
    Combine OU-based Z-score with a momentum filter:
      Zscore = (residual - mu_resid)/sigma
      Momentum = (P_t - P_{t-5})/P_{t-5}

    Trading Rules:
      - If Zscore > +z_thresh AND momentum < mom_thresh => short
      - If Zscore < -z_thresh AND momentum > -mom_thresh => long
      - Else => flat
    """
    df_bt = pd.DataFrame(index=price_series.index)
    df_bt["price"] = price_series.values
    df_bt["returns"] = df_bt["price"].pct_change().fillna(0.0)

    # Z-score
    zscore = (residual - mu_resid)/sigma
    df_bt["zscore"] = zscore

    # Momentum
    mom = compute_momentum(price_series, window=mom_window)
    df_bt["momentum"] = mom.fillna(0.0)

    position = np.zeros(len(df_bt))
    for i in range(len(df_bt)):
        z = df_bt["zscore"].iloc[i]
        mo = df_bt["momentum"].iloc[i]
        # Overbought => short if momentum not too bullish
        if z > z_thresh and mo < mom_thresh:
            position[i] = -1
        # Oversold => long if momentum not too bearish
        elif z < -z_thresh and mo > -mom_thresh:
            position[i] = 1
        else:
            position[i] = 0

    df_bt["position"] = position

    df_bt["strategy_pnl"] = df_bt["position"].shift(1)*df_bt["returns"]
    df_bt.loc[df_bt.index[0], "strategy_pnl"] = 0.0

    df_bt["strategy_cumret"] = (1.0 + df_bt["strategy_pnl"]).cumprod()
    df_bt["buyhold_pnl"] = df_bt["returns"]
    df_bt["buyhold_cumret"] = (1.0 + df_bt["buyhold_pnl"]).cumprod()

    # Also store the zscore for final day usage
    return df_bt

#############################################################################
# 5. Performance Metrics
#############################################################################
def compute_performance_stats(pnl_series, freq_per_year=252):
    """
    Given daily (or each-step) pnl, compute:
      - annualized return
      - annualized volatility
      - sharpe ratio
    """
    pnl_series = pnl_series.dropna()
    daily_mean = pnl_series.mean()
    daily_std  = pnl_series.std(ddof=1)

    ann_return = (1 + daily_mean)**freq_per_year - 1
    ann_vol    = daily_std*np.sqrt(freq_per_year)
    if ann_vol > 1e-10:
        sharpe = ann_return/ann_vol
    else:
        sharpe = np.nan
    return ann_return, ann_vol, sharpe

#############################################################################
# 6. Current Signal & Forecast
#############################################################################
def compute_current_signal(price_series, intercept, slope, mu_resid, sigma, kappa, z_thresh=2.0):
    """
    - price_series: The entire historical 'Adj Close'.
    - intercept, slope: from Trend fit
    - mu_resid, sigma, kappa: from OU residual fit
    - z_thresh: Z-score threshold

    We'll assume the last day is our 'current day'.
    Compute the current Z-score:
       z_current = (Residual_current - mu_resid)/sigma
    If z_current > +z_thresh => "SHORT"
    If z_current < -z_thresh => "LONG"
    Else => "NONE"

    Return:
      - current_signal (LONG/SHORT/NONE)
      - revert_price
      - time_to_half (half-life)
      - time_to_90pct (ln(10)/kappa)
      - expected_pct_return
      - last_date
    """

    # Last day index & price
    last_date = price_series.index[-1]
    current_price = price_series.iloc[-1]
    N = len(price_series)

    # Current trend estimate
    current_trend = intercept + slope*(N-1)  # t_idx = N-1
    # Residual_current = P_{last} - (trend_{last})
    residual_current = current_price - current_trend
    # Z-score
    z_current = (residual_current - mu_resid)/sigma

    # Basic reversion times
    half_life = np.log(2)/kappa if kappa>1e-10 else np.nan
    t_90pct   = np.log(10)/kappa if kappa>1e-10 else np.nan

    # revert_price = current_trend + mu_resid
    revert_price = current_trend + mu_resid

    # Determine signal
    if z_current > z_thresh:
        current_signal = "SHORT"
    elif z_current < -z_thresh:
        current_signal = "LONG"
    else:
        current_signal = "NONE"

    # Expected % return from current_price to revert_price:
    if current_signal == "LONG":
        expected_pct_return = (revert_price - current_price)/current_price * 100
    elif current_signal == "SHORT":
        expected_pct_return = (current_price - revert_price)/current_price * 100
    else:
        expected_pct_return = 0.0

    # For "last signal date," if current day is a signal then use it
    last_signal_date = None
    if abs(z_current) >= z_thresh:
        last_signal_date = last_date
    else:
        # (Advanced search can be implemented if needed)
        pass

    # Expected closing dates based on reversion times
    expected_closing_date_half = pd.NaT
    expected_closing_date_90   = pd.NaT
    if pd.notnull(half_life):
        expected_closing_date_half = last_date + pd.Timedelta(days=half_life)
    if pd.notnull(t_90pct):
        expected_closing_date_90 = last_date + pd.Timedelta(days=t_90pct)

    return {
        "last_date": str(last_date.date()),
        "z_current": z_current,
        "current_signal": current_signal,
        "revert_price": revert_price,
        "time_to_half_revert": half_life,
        "time_to_90pct_revert": t_90pct,
        "expected_pct_return": expected_pct_return,
        "last_signal_date": str(last_signal_date.date()) if last_signal_date else None,
        "expected_date_half": str(expected_closing_date_half.date()) if expected_closing_date_half is not pd.NaT else None,
        "expected_date_90": str(expected_closing_date_90.date()) if expected_closing_date_90 is not pd.NaT else None
    }

#############################################################################
# 7. Analyze Ticker
#############################################################################
def analyze_ticker(ticker, api_key, from_date, to_date,
                   z_thresh=2.0, mom_window=5, mom_thresh=0.0):
    """
    1) Fetch 2y daily data for `ticker`.
    2) Trend+OU residual analysis
    3) If OU is valid, backtest the momentum-filtered MR strategy
    4) Compute a 'current signal' & forecast
    5) Return a summary dict with details
    """
    result = {"ticker": ticker}
    try:
        df = get_price_data_fmp(ticker, api_key, from_date, to_date)
        price_series = df["Adj Close"].copy()
        if len(price_series) < 30:
            result["error"] = "Not enough data points."
            return result

        # Trend+OU
        model_res = detrend_and_fit_ou(price_series)
        result["trend_intercept"] = model_res["intercept"]
        result["trend_slope"] = model_res["slope"]
        result["trend_r2"] = model_res["r2_trend"]
        result["data_points"] = len(price_series)
        result["start_date"] = str(price_series.index[0].date())
        result["end_date"]   = str(price_series.index[-1].date())

        if "ou_error" in model_res:
            result["ou_error"] = model_res["ou_error"]
            return result

        kappa    = model_res["ou_kappa"]
        mu_resid = model_res["ou_mu"]
        sigma    = model_res["ou_sigma"]
        hl       = model_res["ou_half_life"]

        result["ou_kappa"] = kappa
        result["ou_mu"] = mu_resid
        result["ou_sigma"] = sigma
        result["ou_half_life"] = hl

        # Backtest
        residual = model_res["residual"]
        df_bt = backtest_mr_with_momentum(price_series, residual, mu_resid, sigma,
                                          z_thresh=z_thresh,
                                          mom_window=mom_window,
                                          mom_thresh=mom_thresh)

        # Performance Metrics
        strat_annr, strat_vol, strat_sharpe = compute_performance_stats(df_bt["strategy_pnl"])
        bh_annr, bh_vol, bh_sharpe = compute_performance_stats(df_bt["buyhold_pnl"])

        result["strategy_ann_return"] = strat_annr
        result["strategy_ann_vol"]    = strat_vol
        result["strategy_sharpe"]     = strat_sharpe
        result["buyhold_ann_return"]  = bh_annr
        result["buyhold_ann_vol"]     = bh_vol
        result["buyhold_sharpe"]      = bh_sharpe

        # Current Signal & Forecast
        current_sig_info = compute_current_signal(price_series,
                                                  model_res["intercept"],
                                                  model_res["slope"],
                                                  mu_resid,
                                                  sigma,
                                                  kappa,
                                                  z_thresh=z_thresh)

        result.update(current_sig_info)

    except Exception as e:
        result["error"] = str(e)
    return result

#############################################################################
# 8. MAIN Loop over S&P 500 using Parallel Processing and Save to Google Drive
#############################################################################
def main():
    api_key   = os.environ.get("API_KEY")  # <-- Replace with your valid FMP API key
    days_hist = 730  # 2 years
    to_date   = datetime.today().strftime("%Y-%m-%d")
    from_date = (datetime.today() - timedelta(days=days_hist)).strftime("%Y-%m-%d")

    sp500_tickers = get_sp500_tickers()
    print(f"Found {len(sp500_tickers)} S&P 500 tickers.")

    # Use parallel processing with 8 CPU cores
    func = partial(analyze_ticker, api_key=api_key, from_date=from_date, to_date=to_date,
                   z_thresh=2.0, mom_window=5, mom_thresh=0.0)
    with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
        results = list(executor.map(func, sp500_tickers))

    # Print results in the original ticker order
    for i, (ticker, res) in enumerate(zip(sp500_tickers, results), start=1):
        print(f"\n[{i}/{len(sp500_tickers)}] Analyzing {ticker} ...")
        if "error" in res:
            print(f"  Error: {res['error']}")
        else:
            pts = res.get("data_points", 0)
            sd = res.get("start_date", "?")
            ed = res.get("end_date", "?")
            print(f"  Data Points: {pts} from {sd} to {ed}")
            if "ou_error" in res:
                print(f"  OU Error => {res['ou_error']}")
            else:
                kappa  = res["ou_kappa"]
                hl     = res["ou_half_life"]
                s_annr = res["strategy_ann_return"]*100
                s_shp  = res["strategy_sharpe"]
                b_annr = res["buyhold_ann_return"]*100
                b_shp  = res["buyhold_sharpe"]
                print(f"  OU => kappa={kappa:.4f}, half-life={hl:.2f} days")
                print(f"  Strategy => {s_annr:.2f}% annual, Sharpe={s_shp:.2f}")
                print(f"  Buy&Hold => {b_annr:.2f}% annual, Sharpe={b_shp:.2f}")

                c_sig   = res.get("current_signal", "NONE")
                r_price = res.get("revert_price", 0)
                half_t  = res.get("time_to_half_revert", 0)
                ninty_t = res.get("time_to_90pct_revert", 0)
                exp_ret = res.get("expected_pct_return", 0)
                sig_dt  = res.get("last_signal_date", None)
                half_dt = res.get("expected_date_half", None)
                ninty_dt= res.get("expected_date_90", None)
                print(f"  Current Signal: {c_sig}")
                if c_sig != "NONE":
                    print(f"    Revert Price: {r_price:.2f}")
                    print(f"    Time to 1/2 revert: {half_t:.1f} days (=> {half_dt})")
                    print(f"    Time to 90% revert: {ninty_t:.1f} days (=> {ninty_dt})")
                    print(f"    Expected % Return: {exp_ret:.2f}%")
                    print(f"    Last Signal Date: {sig_dt if sig_dt else 'N/A'}")

    df_res = pd.DataFrame(results)


    base_folder = "/workspaces/Stock-Market-Prediction/Quantative Investment/QI Output/Reversion"
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    dated_folder = os.path.join(base_folder, datetime.today().strftime("%Y-%m-%d"))
    if not os.path.exists(dated_folder):
        os.makedirs(dated_folder)
    output_file = os.path.join(dated_folder, "sp500_mr_momentum_results_with_signal.csv")
    df_res.to_csv(output_file, index=False)
    print(f"\nDone. Results saved to '{output_file}'.")

if __name__ == "__main__":
    main()


Found 503 S&P 500 tickers.

[1/503] Analyzing MMM ...
  Data Points: 500 from 2023-05-19 to 2025-05-16
  OU => kappa=0.0252, half-life=27.54 days
  Strategy => 4.67% annual, Sharpe=0.29
  Buy&Hold => 48.71% annual, Sharpe=1.48
  Current Signal: SHORT
    Revert Price: 147.69
    Time to 1/2 revert: 27.5 days (=> 2025-06-12)
    Time to 90% revert: 91.5 days (=> 2025-08-15)
    Expected % Return: 3.54%
    Last Signal Date: 2025-05-16

[2/503] Analyzing AOS ...
  Data Points: 500 from 2023-05-19 to 2025-05-16
  OU => kappa=0.0132, half-life=52.42 days
  Strategy => 15.03% annual, Sharpe=1.05
  Buy&Hold => 6.36% annual, Sharpe=0.27
  Current Signal: LONG
    Revert Price: 74.84
    Time to 1/2 revert: 52.4 days (=> 2025-07-07)
    Time to 90% revert: 174.1 days (=> 2025-11-06)
    Expected % Return: 6.12%
    Last Signal Date: 2025-05-16

[3/503] Analyzing ABT ...
  Data Points: 500 from 2023-05-19 to 2025-05-16
  OU => kappa=0.0221, half-life=31.43 days
  Strategy => 3.05% annual, Sharp